In [17]:
#!/usr/bin/env python

#environment setting
import pandas as pd
import sqlite3
import re
# Library Import and Environment Setting needed for Logistic Regression

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

plt.rc("font", size=16)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os
import os.path
import glob
import re

import json
import time

from collections import defaultdict
from collections import Counter

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk.data
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('stopwords')

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

! pip install profanity-check
from profanity_check import predict, predict_prob
# https://github.com/vzhou842/profanity-check


import argparse

!pip install coverage

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alexpapaioannou/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexpapaioannou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 184kB 5.3MB/s 
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
%time
    
#     parser = argparse.ArgumentParser('Parses the given directory of lyrics')
#     parser.add_argument('dir_given', help='Directory of the lyrics')
#     args = parser.parse_args()
#     dir_given = args.dir_given


#Simulated 'Given' directory, needs to be changed to input from command line kss0416
dir_given = r'/Users/alexpapaioannou/Documents/GitHub/TFA_Project/Lyrics'

start_time = time.time()

#Need to change path below to start in lyrics directory kss0416
os.chdir(dir_given)

raw_filenames = []
artists = []

# Get .txt files names
# Changed below line to use the lyrics directory as the current directory kss0416
for file_name in os.listdir():
    if file_name.endswith('.txt'):
        raw_filenames.append(file_name)
        artists.append(file_name.split("~")[2].split(".txt")[0])

def raw_filenames_list(str)->list:
    raw_filenames_list_ = []
    for filename in os.listdir('Lyrics'):
        if filename.endswith(str):
            raw_filenames_list_.append(filename)
    return raw_filenames_list_

def artists_list(str)->list:
    artists_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            artists_list_.append(filename.split("~")[2].split(".txt")[0])
    return artists_list_

def songs_list(str)->list:
    songs_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            songs_list_.append(filename.split("~")[1].split(".txt")[0])
    return songs_list_

def song_ordering_numbers_list(str)->list:
    song_ordering_numbers_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            song_ordering_numbers_list_.append(filename.split("~")[0].split(".txt")[0])
    return song_ordering_numbers_list_

def artist_s_songs_list(str)->list:
    artist_s_songs_list_ = []
    for raw_filename in os.listdir():
        if raw_filename.endswith(str+'.txt'):
            artist_s_songs_list_.append(raw_filename)
    return artist_s_songs_list_

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


IndexError: list index out of range

In [39]:
%time
#Changed to function kss0416
def song_cleaning(artists):
    #Creating the Cleaned_Songs Directory kss0416
    try:
        os.mkdir(dir_given + '/Cleaned_Songs')
    except FileExistsError:  
        pass

    for artist in set(artists):
        for song in artist_s_songs_list(artist):
    #Changed below to base on single given directory kss0416
            f = open(dir_given + r'/' +  song, 'rb')
            all_words = ''
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                all_words += this_sentence
            #remove identifiers like chorus, verse, etc
            all_words = re.sub(r'[\(\[],.*?[\)\]]', '', all_words)
            #remove empty lines
            all_words = os.linesep.join([s for s in all_words.splitlines() if s])
            f.close()
            #Deleted Directory variable and modified below line to take given directory
            f = open(os.path.join(dir_given + '/Cleaned_Songs', 'cleaned_' + song ), "wb")
            f.write(all_words.encode('utf-8'))
            f.close()
            
song_cleaning(artists)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [38]:
%time
plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0
            
def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    #Modified below line to use dir_given
    for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs/'):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_

def word_num(artists):
    print('running')
    i = 0
    for artist in set(artists):
        num_words = 0
        all_words = ''
        df.loc[i] = (artist, num_words)
        i += 1
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this

word_num(artists)        

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
running


In [6]:
cell4s_t = time.time()

# #Adjusted below directory, and made function kss0426
# os.chdir(dir_given)

def sentiment(artists):
    df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
    #Added download for vader lexicon in the import section for below step kss0426
    sid = SentimentIntensityAnalyzer()
    # Commas do not affect the SentimentIntensityAnalyzer() (http://www.nltk.org/howto/sentiment.html)
    i=0

    num_neutral = 0
    num_negative = 0
    num_positive = 0

    for artist in set(artists):

        num_neutral = 0
        num_negative = 0
        num_positive = 0

        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                comp = sid.polarity_scores(this_sentence)
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > -0.5 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1

        num_total = num_negative + num_neutral + num_positive
        if num_total != 0:
            percent_negative = (num_negative/float(num_total))*100
            percent_neutral = (num_neutral/float(num_total))*100
            percent_positive = (num_positive/float(num_total))*100
            df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
            i+=1
        else:
            percent_negative = 0
            percent_neutral = 0
            percent_positive = 0
            df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
            i+=1
    
sentiment(artists)

cell4_t = time.time()
print(cell4_t - cell4s_t)

47.14235520362854


In [7]:
cell5s_t = time.time()

plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0

def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs'):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_

def min_max_numfunc(artists):
    num_words_list_ = []

    for artist in set(artists):
        num_words = 0
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
        num_words_list_.append(num_words)
        
    min_num_words_list_ = min(num_words_list_)
    max_num_words_list_ = max(num_words_list_)
    return min_num_words_list_, max_num_words_list_
    
min_num_words_list_, max_num_words_list_ = min_max_numfunc(artists)

2.853158712387085


In [8]:
def regularize_num(artists):
    for artist in set(artists):
        num_words = 0
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
            regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
            num_words_score = 1*regularization_step + 0*(1-regularization_step)
            num_words_score = round(num_words_score,2)
    
    return num_words_score

num_words_score = regularize_num(artists)

2.5900981426239014


In [11]:
def lexical_richness(artists):
    plt.rcParams['figure.figsize'] = (126,190)

    df = pd.DataFrame(columns=('artist', 'lexicalrichness'))
    for artist in set(artists):
        num_words = 0
        all_words = ''
        raw_text = ""
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                raw_text += this_sentence

        words = raw_text.split(" ")
        filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

        a = len(set(filtered_words))
        b = len(words)

lexical_richness(artists)

def unique_words(artists):
    unique_words_list_ = []

    for artist in set(artists):
        num_words = 0
        all_words = ''
        raw_text = ""    
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                raw_text += this_sentence
        words = raw_text.split(" ")
        filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

        a = len(set(filtered_words))
        b = len(words)
        unique_words_ratio = a/float(b)
        unique_words_ratio = round(unique_words_ratio,2)
        unique_words_list_.append(unique_words_ratio)

    return unique_words_list_
  
unique_words(artists)

58.3951370716095


In [ ]:
cell8s_t = time.time()

num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]

love_ratio_list_ = []
dict_for_json = {}

with open('./data_output.txt', 'w') as f:

    for artist in artists:   

        num_neutral = 0
        num_negative = 0
        num_positive = 0
        
        num_words = 0
        all_words = ''
        raw_text = "" 
        
        counter_for_love_words = 0

        for song in artist_s_cleaned_songs_list(artist):
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
min_love_ratio_list_ = min(love_ratio_list_)
max_love_ratio_list_ = max(love_ratio_list_)

# print(love_ratio_list_)      # uncomment to see the list with the number of words per song
# print(min(love_ratio_list_)) # uncomment to see the minimum of the list with the number of words per song
# print(max(love_ratio_list_)) # uncomment to see the maximum of the list with the number of words per song

# Creating the regularization step for the length attribute

for artist in set(artists):   

    num_neutral = 0
    num_negative = 0
    num_positive = 0

    num_words = 0
    all_words = ''
    raw_text = "" 

    counter_for_love_words = 0
    
    for song in artist_s_cleaned_songs_list(artist):
        cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
        id_ = song.split("cleaned_")[1].split("~")[0]
        f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')            
            raw_text += this_sentence
        words = raw_text.split(" ")
        for item in words:
            if item in love_words_list_:
                counter_for_love_words += 1
                len_counter_for_love_words = len(love_words_list_)
                love_ratio = counter_for_love_words/float(len_counter_for_love_words)
                love_ratio = round(love_ratio,4)
                love_ratio_list_.append(love_ratio)
            
        regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
        love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
        love_ratio_score = round(love_ratio_score,2)
#         print(love_ratio_score) # uncomment to see the complexity of each song

cell8_t = time.time()
print(cell8_t - cell8s_t)

In [36]:
cell9s_t = time.time()

df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0

# os.chdir('/Users/alexpapaioannou/Dropbox/Personal Things/Studies Related/NY/COLUMBIA UNIVERSITY/IEOR/MSMSE/Courses/IEOR 4501 E 001 TOOLS FOR ANALYTICS/TOOLS_2019/Project_2019/')
os.chdir(dir_given)
num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]

love_ratio_list_ = []
dict_for_json = {}
with open('data_output.txt', 'w') as fp:
# with open('./data_output.txt', 'w') as f:
    print('{\n      "characterizations": [')
    print('\n             {')
    for artist in set(artists):   

        num_neutral = 0
        num_negative = 0
        num_positive = 0
        
        num_words = 0
        all_words = ''
        raw_text = "" 
        
        counter_for_love_words = 0


#         if len(artist_s_cleaned_songs_list(artist)) == 1:
        
        for song in artist_s_cleaned_songs_list(artist):
            dict_for_json_song = {}
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            
            dict_for_json_song["id"] = id_
            dict_for_json_song["artist"] = artist
            dict_for_json_song["title"] = cleaned_song_name
            
            print(f'\n                     "id" = {id_},\n                     "artist" = {artist},') 

            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
                
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
                
                comp = sid.polarity_scores(this_sentence)
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > 0 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1
                
            sentence_for_profanity_check_ = [this_sentence]
            profanity_check = predict_prob(sentence_for_profanity_check_)
            profanity_score = float(' '.join(map(str, profanity_check)))
            dict_for_json_song["kids_safe"] = round(profanity_score,2)
                
            regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
            num_words_score = 1*regularization_step + 0*(1-regularization_step)
            num_words_score = round(num_words_score,2)

            num_total = num_negative + num_neutral + num_positive
            if num_total != 0:
                percent_negative = round((num_negative/float(num_total))*100,2)
                percent_neutral = round((num_neutral/float(num_total))*100,2)
                percent_positive = round((num_positive/float(num_total))*100,2)
                df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
                i+=1
            else:
                percent_negative = 0
                percent_neutral = 0
                percent_positive = 0
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            a = len(set(filtered_words))
            b = len(words)
            unique_words_ratio = a/float(b)
            unique_words_ratio = round(unique_words_ratio,2)
            unique_words_list_.append(unique_words_ratio)
            
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
            regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_ - 16) 
            love_ratio_score = 1*regularization_step
            love_ratio_score = round(love_ratio_score,2)
            
            dict_for_json_song["love"] = round(love_ratio_score,2)
            dict_for_json_song["mood"] = round(comp,2)
            dict_for_json_song["length"] = num_words_score
            dict_for_json_song["complexity"] = unique_words_ratio
            

#             dict_for_json.setdefault('characterizations:', []).append(dict_for_json_song)

{
      "characterizations": [

             {

                     "id" = 25,
                     "artist" = Young Gunz,

                     "id" = 406,
                     "artist" = Natalie Cole,

                     "id" = 636,
                     "artist" = Jerry Lee Lewis,

                     "id" = 620,
                     "artist" = Jerry Lee Lewis,

                     "id" = 240,
                     "artist" = The Jeff Healey Band,

                     "id" = 926,
                     "artist" = Oscar Peterson,

                     "id" = 99,
                     "artist" = Oscar Peterson,

                     "id" = 163,
                     "artist" = The Band,

                     "id" = 881,
                     "artist" = Rock Goddess,

                     "id" = 945,
                     "artist" = Altan,

                     "id" = 306,
                     "artist" = Lightnin' Hopkins,

                     "id" = 863,
                     "artist" =

In [37]:
# print(dict_for_json)
print(json.dumps(dict_for_json, indent=4))

{}


In [30]:
# with open('data_output.json', 'r') as f:  
#     dict_for_json = json.load(f)
#     print(dict_for_json)

len(dict_for_json['characterizations:'])

838

In [ ]:
# def love_ratio_score(song)->float:
    
#     words = []
#     raw_text = "" 
#     counter_for_love_words = 0
    
#     love_words_list_ = [
#                    'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
# #                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
# #                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
# #                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
#                    'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
#                    'intimacy', 'joy', 'kiss', 'kisses', 
#                    'love', 'loves', 'loved', 'loving', 
# #                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
# #                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
#                     ]
    
#     love_ratio_list_ = []

#     f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#     for sentence in f.readlines():
#         this_sentence = sentence.decode('utf-8')
#         raw_text += this_sentence

#     words = raw_text.split(" ")
#     filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#     for item in words:
#         if item in love_words_list_:
#             counter_for_love_words += 1
#     len_counter_for_love_words = len(love_words_list_)
#     love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#     love_ratio = round(love_ratio,4)
#     love_ratio_list_.append(love_ratio)
    
#     for artist in set(artists):   
        
#         for song in artist_s_cleaned_songs_list(artist):
#             cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#             id_ = song.split("cleaned_")[1].split("~")[0]
#             f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
                
#                 raw_text += this_sentence
            
#             words = raw_text.split(" ")
#             filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
#             for item in words:
#                 if item in love_words_list_:
#                     counter_for_love_words += 1
#             len_counter_for_love_words = len(love_words_list_)
#             love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#             love_ratio = round(love_ratio,4)
#             love_ratio_list_.append(love_ratio) 
            
#     min_love_ratio_list_ = min(love_ratio_list_)
#     max_love_ratio_list_ = max(love_ratio_list_)

#     regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
#     love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
#     love_ratio_score = round(love_ratio_score,1)
#     return love_ratio_score

In [ ]:
# a = love_ratio_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')
# print(a)